<a href="https://colab.research.google.com/github/HackWarts-2/app/blob/main/Weaviate_Ingest_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [1]:
%pip install --upgrade --quiet weaviate-client apify langchain sentence_transformers langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.3/368.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379

In [16]:
from google.colab import userdata

WEAVIATE_URL = userdata.get("WEAVIATE_URL")
WEAVIATE_API_KEY = userdata.get("WEAVIATE_API_KEY")
HUGGINGFACE_KEY = userdata.get("HUGGINGFACE_KEY")
apify_api_key = userdata.get("APIFY_API_KEY")
google_api_key = userdata.get("GOOGLE_SEARCH_API")
search_engine_id = userdata.get("SEARCH_ENGINE_ID")

In [3]:
import weaviate
from weaviate.classes.init import Auth
import os
from apify_client import ApifyClient
import requests
import json
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from weaviate.classes.query import MetadataQuery
from weaviate.classes.query import Filter
from weaviate.classes.config import Configure, Property, DataType, Tokenization
import weaviate.classes as wvc

In [4]:
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print(client.is_ready())

True


#Define function to fetch data

In [5]:
def getPosts(url):
    # Initialize the ApifyClient with API token
    client = ApifyClient(apify_api_key)

    # Prepare the Actor input
    run_input = {
        "addParentData": False,
        "directUrls": [url],
        "enhanceUserSearchWithFacebookPage": True,
        "isUserReelFeedURL": True,
        "isUserTaggedFeedURL": False,
        "resultsLimit": 50,
        "resultsType": "posts",
    }

    posts = []

    # Run the Actor and wait for it to finish
    run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        posts.append(item)
    print("Length of posts: " + str(len(posts)))

    def extractPostData(original_dict):
        # Extract the necessary information from the original dictionary
        extracted_info = {
            'type': original_dict.get('type'),
            'timestamp': original_dict.get('timestamp'),
            'caption': original_dict.get('caption'),
            'hashtags': ' '.join(['#' + tag for tag in original_dict.get('hashtags') or []]),
            'mentions': ' '.join(['@' + mention for mention in original_dict.get('mentions') or []]),
            'url': original_dict.get('url'),
            'videoUrl': original_dict.get('videoUrl') if original_dict.get('type') == 'Video' else None,
            'displayUrl': original_dict.get('displayUrl'),
            'commentsCount': original_dict.get('commentsCount'),
            'likesCount': original_dict.get('likesCount'),
            'videoViewCount': original_dict.get('videoViewCount') if original_dict.get('type') == 'Video' else None,
            'videoPlayCount': original_dict.get('videoPlayCount') if original_dict.get('type') == 'Video' else None,
            'ownerUsername': original_dict.get('ownerUsername'),
            'taggedUsers': ' '.join(['@' + user['username'] for user in original_dict.get('taggedUsers', [])]),
            'latestComments': '\n'.join([comment['text'] for comment in original_dict.get('latestComments', [])])
        }

        # Create a content string to include all the extracted details
        content = f"""type: {extracted_info['type']}
        timestamp: {extracted_info['timestamp']}
        caption: {extracted_info['caption']}
        hashtags: {extracted_info['hashtags']}
        mentions: {extracted_info['mentions']}
        commentsCount: {extracted_info['commentsCount']}
        likesCount: {extracted_info['likesCount']}
        videoViewCount: {extracted_info['videoViewCount']}
        videoPlayCount: {extracted_info['videoPlayCount']}
        ownerUsername: {extracted_info['ownerUsername']}
        taggedUsers: {extracted_info['taggedUsers']}
        latestComments: {extracted_info['latestComments']}
        """

        post_url = extracted_info['url']  # Extract the post's URL

        return content, post_url

    extracted_posts = []
    for post in posts:
        extracted_post, post_url = extractPostData(post)
        extracted_posts.append({"profileURL": url, "postURL": post_url, "content": extracted_post})
    return extracted_posts

def getSimilarProfiles(query):
    searchQuery = query
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": searchQuery,
        "key": google_api_key,
        "cx": search_engine_id,
        "num": 5
    }

    response = requests.get(url, params=params)
    results = response.json()
    links_array = [item['link'] for item in results['items']]
    print(links_array)

    all_posts_data = []

    for link in links_array:
        print(link)
        posts_data = getPosts(link)
        all_posts_data.extend(posts_data)  # Combine all posts into one list
    return all_posts_data

#Create embeddings of fetched data

In [6]:
def add_embeddings_to_posts(posts_data):
    """
    Adds vector embeddings to each post's content.

    Args:
        posts_data (list of dicts): List of dictionaries with keys: profileURL, postURL, content.

    Returns:
        list of dicts: Updated list with an added "vector" key containing the embeddings.
    """
    # Initialize the Hugging Face Embedding class
    embeddings_model = HuggingFaceEmbeddings()

    vectors = []

    for post in posts_data:
        content = post['content']
        # Generate embeddings for the content
        vector = embeddings_model.embed_query(content)
        # Add the vector embeddings to the dictionary
        vectors.append(vector)

    return vectors

#Functions to vectorise search query and perform vector search

In [7]:
def convert_text_to_embeddings(input_text):
    """
    Converts an input string to vector embeddings.

    Args:
        input_text (str): The input string to convert.

    Returns:
        list of floats: The vector embeddings for the input text.
    """
    # Initialize the Hugging Face Embedding class
    embeddings_model = HuggingFaceEmbeddings()

    # Generate embeddings for the input text
    vector_embeddings = embeddings_model.embed_query(input_text)

    return vector_embeddings

In [8]:
def vector_search(query, collection):

  collection = client.collections.get(collection)

  query_vector = convert_text_to_embeddings(query)

  response = collection.query.near_vector(
      near_vector=query_vector,
      limit=10,
      return_metadata=MetadataQuery(distance=True)
  )
  result = []
  for o in response.objects:
      print(o.properties)
      result.append(o.properties)
      print(o.metadata.distance)
  return result

#Function to create collection

In [9]:
def create_collection(name):
  client.collections.create(
      name,
      vectorizer_config=wvc.config.Configure.Vectorizer.none(),

      properties=[
          Property(
              name="profileURL",
              data_type=DataType.TEXT,
              skip_vectorization=True,
              tokenization=Tokenization.FIELD
          ),
          Property(
              name="postURL",
              data_type=DataType.TEXT,
              skip_vectorization=True,
              tokenization=Tokenization.FIELD
          ),
          Property(
              name="content",
              data_type=DataType.TEXT,
              tokenization=Tokenization.WHITESPACE
          ),
      ]
  )

#Function to get and ingest profiles data based on search query

Creating functions to save extracted data with corresponding embeddings to a json file for later ingestion so as not to spend so much time again

(note to self: did this after ingestion till Lifestyle had completed so will have to redo until then)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def save_data_to_file(data, file_name):
    with open(file_name, 'w') as file:
        json.dump(data, file)

def load_data_from_file(file_name):
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            data = json.load(file)
        return data
    else:
        return None

In [12]:
def ingest_data(category, collection):
  file_name = f"/content/drive/MyDrive/Insta_Data/data_{category}_{collection}.json"

  # Check if data already exists in a file
  data = load_data_from_file(file_name)
  if data is None:
      # If data does not exist, fetch and save it
      data = getSimilarProfiles(category)
      vectors = add_embeddings_to_posts(data)

      # Save the data to a file
      save_data_to_file(data, file_name)
      save_data_to_file(vectors, file_name.replace('.json', '_vectors.json'))
  else:
      # Load vectors if data is already available
      vectors = load_data_from_file(file_name.replace('.json', '_vectors.json'))

  collection = client.collections.get(collection)

  with collection.batch.dynamic() as batch:
      for i, data_row in enumerate(data):
          batch.add_object(
              properties=data_row,
              vector=vectors[i]
          )

In [13]:
# collections = ["Beauty", "Fashion", "Food", "Fitness", "Travel", "Lifestyle", "Photography", "Art", "Tech", "Business_Finance", "Education", "Entertainment", "Automotive", "Pets", "Gardening", "Home", "Religion", "Environment", "Gaming", "Events", "Misc", "Books"]
# for collection in collections:
#   create_collection(collection)

In [14]:
#ingestion_data = {"Fashion":["fashion blogger", "fashion influencer", "fashion small business", "clothing business", "clothing & accessories", "luxury fashion"], "Food": ["food blogger", "recipe", "food", "restaurant", "cafe", "food business", "food home business", "street food", "Healthy Eating", "Vegan", "Vegetarian", "clean eating", "Desserts" ,"Cakes", "Cookies", "Brownies", "Beverages", "Coffee", "Tea", "Food Photography", "coffee business"], "Fitness": ["Fitness", "Fitness Coach", "Yoga", "yoga instructor", "Nutritionist", "Dietitian", "Weight Loss", "Bodybuilding", "Mental Health", "Wellness", "Home Workouts"], "Travel": ["Travel Blogger", "Adventure", "Travel", "Solo Travel", "Travel Photography", "tourism", "tourist", "travel enthusiast", "globe trotter"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Sustainable Living", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching", "art account", "art blogger"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance" "Gaming", "coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency", "tech reviews"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "Personal Finance", "E-commerce", "Stock Market","Freelancing", "Leadership & Management"], "Education": ["Language Learning", "Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "career blog", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Book Reviews", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Religion": ["islam", "christianity", "jewish blogger", "muslim blogger", "muslim influencer", "christrian blogger", "religion blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming", "gaming news", "mobile gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "booktok", "book recommendations", "reader"]}


#switched to GPU after the first 3 ingestions bc taking too long

#ingestion_data = {"Fashion":["clothing business", "clothing & accessories", "luxury fashion"], "Food": ["food blogger", "recipe", "food", "restaurant", "cafe", "food business", "food home business", "street food", "Healthy Eating", "Vegan", "Vegetarian", "clean eating", "Desserts" ,"Cakes", "Cookies", "Brownies", "Beverages", "Coffee", "Tea", "Food Photography", "coffee business"], "Fitness": ["Fitness", "Fitness Coach", "Yoga", "yoga instructor", "Nutritionist", "Dietitian", "Weight Loss", "Bodybuilding", "Mental Health", "Wellness", "Home Workouts"], "Travel": ["Travel Blogger", "Adventure", "Travel", "Solo Travel", "Travel Photography", "tourism", "tourist", "travel enthusiast", "globe trotter"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Sustainable Living", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching", "art account", "art blogger"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance" "Gaming", "coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency", "tech reviews"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "Personal Finance", "E-commerce", "Stock Market","Freelancing", "Leadership & Management"], "Education": ["Language Learning", "Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "career blog", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Book Reviews", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Religion": ["islam", "christianity", "jewish blogger", "muslim blogger", "muslim influencer", "christrian blogger", "religion blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming", "gaming news", "mobile gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "booktok", "book recommendations", "reader"]}

#apify key rate reached :/

#ingestion_data = {"Food": ["recipe", "food", "restaurant", "cafe", "food business", "food home business", "street food", "Healthy Eating", "Vegan", "Vegetarian", "clean eating", "Desserts" ,"Cakes", "Cookies", "Brownies", "Beverages", "Coffee", "Tea", "Food Photography", "coffee business"], "Fitness": ["Fitness", "Fitness Coach", "Yoga", "yoga instructor", "Nutritionist", "Dietitian", "Weight Loss", "Bodybuilding", "Mental Health", "Wellness", "Home Workouts"], "Travel": ["Travel Blogger", "Adventure", "Travel", "Solo Travel", "Travel Photography", "tourism", "tourist", "travel enthusiast", "globe trotter"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Sustainable Living", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching", "art account", "art blogger"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance" "Gaming", "coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency", "tech reviews"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "Personal Finance", "E-commerce", "Stock Market","Freelancing", "Leadership & Management"], "Education": ["Language Learning", "Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "career blog", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Book Reviews", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Religion": ["islam", "christianity", "jewish blogger", "muslim blogger", "muslim influencer", "christrian blogger", "religion blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming", "gaming news", "mobile gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "booktok", "book recommendations", "reader"]}

#apify key rate reached again RIP

#ingestion_data = {"Food": ["Desserts" ,"Cakes", "Cookies", "Brownies", "Coffee", "Tea", "Food Photography"], "Fitness": ["Fitness", "Fitness Coach", "Yoga", "yoga instructor", "Nutritionist", "Dietitian", "Weight Loss", "Bodybuilding", "Mental Health", "Wellness", "home workouts"], "Travel": ["Travel Blogger", "Travel", "Solo Travel", "Travel Photography", "tourism","travel enthusiast"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#runtime disconnected :/

#ingestion_data = {"Food": ["Brownies", "Coffee", "Tea", "Food Photography"], "Fitness": ["Fitness", "Fitness Coach", "Yoga", "yoga instructor", "Nutritionist", "Dietitian", "Weight Loss", "Bodybuilding", "Mental Health", "Wellness", "home workouts"], "Travel": ["Travel Blogger", "Travel", "Solo Travel", "Travel Photography", "tourism","travel enthusiast"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#rate limited agaaain

#ingestion_data = {"Fitness": ["home workouts"], "Travel": ["Travel Blogger", "Travel", "Solo Travel", "Travel Photography", "tourism","travel enthusiast"], "Lifestyle": ["Daily Life", "lifestyle blogger", "Home Living", "Organization & Productivity", "DIY & Crafts", "Family & Parenting","Relationship Advice", "Parenting"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Beauty": ["skincare", "makeup", "beauty blogger", "skincare influencer", "nail art"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Beauty": ["nail art"], "Photography": ["Photography",  "Street Photography",  "Food Photography", "Nature Photography", ], "Art": ["Digital Art",  "Painting", "Drawing",  "Abstract Art", "Calligraphy", "Tattoo Art", "Graphic Design", "Art", "Sketching"], "Tech": ["Tech Reviews", "Tech", "tech account", "tech influencer", "data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Tech": ["data scientist", "web developer", "software engineer", "freelance","coding", "programming", "AI", "artificial intelligence", "digital marketing", "crypto", "cryptocurrency"], "Business_Finance": ["Entrepreneurship", "Business Strategy", "Finance", "Investment", "Real Estate", "E-commerce", "Stock Market","Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Business_Finance": ["Freelancing"], "Education": ["Study blog", "Educational Content", "Science", "History & Culture", "Career Advice", "Teaching", "Tutoring", "education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Education": ["education blog"], "Entertainment": ["Musician", "Band", "Actor", "Actress", "Comedian", "Movies", "TV Show", "Celebrity News", "Podcasts","Stand-up Comedy"], "Automotive": ["Car Reviews", "cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#ingestion_data = {"Automotive": ["cars" "Luxury Cars", "Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Vet", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality", "non-profit organization"], "Gaming": ["gamer", "gaming blog", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

#notw: from automotives till events (exclusive), modified function to ingest only 5 posts per user per query so do it again later for more

#ingestion_data = {"Automotive": ["Motorcycles", "racing", "car photography"], "Pets": ["Pet Blogger",  "Cat Lover",  "Wildlife",  "Animal Rescue", "Pet Photography", "Pet Fashion"], "Gardening": ["gardening", "planting", "gardening blogger"], "Home": ["home decor", "interior design", "real estate", "home blogger"], "Environment": ["environment activist", "human rights", "animal rights", "gender equality"], "Gaming": ["gamer", "gaming"], "Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}

ingestion_data = {"Events": ["Event Planning", "Wedding Planning", "Party Decorations", "Event Photography", "Catering", "Wedding Photography"], "Misc": ["memes", "humour", "historical facts", "conspiracy theories", "boardgames", "puzzles", "meme account", "humour"], "Books": ["book blogger", "book review", "writer", "poetry", "book club", "book business", "library", "book photography", "book recommendations"]}



In [17]:
# Iterate over each collection and its corresponding queries
for collection, queries in ingestion_data.items():
    for query in queries:
        # Call the ingest_data function for each query
        ingest_data(query, collection)
        print("Ingested ", query, " into collection ", collection)

['https://www.instagram.com/timelesseventplanning_/', 'https://www.instagram.com/courtneytaylorevents/', 'https://www.instagram.com/somanyheartsnyc/', 'https://www.instagram.com/events_fullcircle/', 'https://www.instagram.com/eventiveexpertplanning/']
https://www.instagram.com/timelesseventplanning_/
Length of posts: 50
https://www.instagram.com/courtneytaylorevents/
Length of posts: 1
https://www.instagram.com/somanyheartsnyc/
Length of posts: 42
https://www.instagram.com/events_fullcircle/
Length of posts: 50
https://www.instagram.com/eventiveexpertplanning/
Length of posts: 8


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

ERROR:weaviate-client:{'message': 'Failed to send 10 objects in a batch of 10. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


Ingested  book review  into collection  Books
['https://www.instagram.com/laurenhissrich/', 'https://www.instagram.com/ahkim.writer/', 'https://www.instagram.com/happywriterpodcast/', 'https://www.instagram.com/cm.writer/', 'https://www.instagram.com/natalie.lampert.writer/']
https://www.instagram.com/laurenhissrich/
Length of posts: 35
https://www.instagram.com/ahkim.writer/
Length of posts: 15
https://www.instagram.com/happywriterpodcast/
Length of posts: 50
https://www.instagram.com/cm.writer/
Length of posts: 50
https://www.instagram.com/natalie.lampert.writer/
Length of posts: 10
Ingested  writer  into collection  Books
['https://www.instagram.com/poetryteatime/', 'https://www.instagram.com/atticuspoetry/', 'https://www.instagram.com/elleneverettpoetry/', 'https://www.instagram.com/thepoetryfox/', 'https://www.instagram.com/quarterlifepoetry/']
https://www.instagram.com/poetryteatime/
Length of posts: 6
https://www.instagram.com/atticuspoetry/
Length of posts: 50
https://www.insta

ERROR:weaviate-client:{'message': 'Failed to send 10 objects in a batch of 10. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


Ingested  poetry  into collection  Books
['https://www.instagram.com/jordys.book.club/', 'https://www.instagram.com/mpbookclub/', 'https://www.instagram.com/kebookclub/', 'https://www.instagram.com/natsbookclub/', 'https://www.instagram.com/spiveysbookclub/']
https://www.instagram.com/jordys.book.club/
Length of posts: 50
https://www.instagram.com/mpbookclub/
Length of posts: 38
https://www.instagram.com/kebookclub/
Length of posts: 1
https://www.instagram.com/natsbookclub/
Length of posts: 22
https://www.instagram.com/spiveysbookclub/
Length of posts: 16
Ingested  book club  into collection  Books
['https://www.instagram.com/whatdavereads/', 'https://www.instagram.com/jodibrandoneditorial/', 'https://www.instagram.com/micaelasbenn/', 'https://www.instagram.com/danmartell/', 'https://instagram.com/download/ios/']
https://www.instagram.com/whatdavereads/
Length of posts: 1
https://www.instagram.com/jodibrandoneditorial/
Length of posts: 50
https://www.instagram.com/micaelasbenn/
Length 

ApifyApiError: Monthly usage hard limit exceeded. Please upgrade your subscription or contact support@apify.com

In [20]:
collection = client.collections.get("Beauty")
i = 0
for item in collection.iterator():
    i += 1
    print(i, item.uuid, item.properties)

1 00309c91-92be-49ac-896a-7a4fb12507ae {'postURL': 'https://www.instagram.com/p/C7M7ZMbxb84/', 'content': "type: Video\n        timestamp: 2024-05-20T19:28:00.000Z\n        caption: I always have to ask first 🤷\u200d♀️ \n\nJokes aside, my doors are open for anyone 🤍\n\n👩🏻\u200d⚕️ 𝗔𝗹𝗹𝗶𝗲 𝗠𝗰𝗔𝗹𝗹𝗶𝘀𝘁𝗲𝗿, 𝗡𝗣-𝗖, 𝗗𝗖𝗡𝗣\n📲 @theatlantainjector\n💉 Botox | Filler | Sculptra | Kybella | PRP | Skincare\n💻 www.atlantainjector.com\u2060\n👩\u200d💻 Book online using #linkinbio ⇨ @theatlantainjector\n\n#theatlantainjector #sculptra #masterinjector #atlantaaesthetician\n        hashtags: #linkinbio #theatlantainjector #sculptra #masterinjector #atlantaaesthetician\n        mentions: @theatlantainjector\n        commentsCount: 30\n        likesCount: 201\n        videoViewCount: 17951\n        videoPlayCount: 40419\n        ownerUsername: theatlantainjector\n        taggedUsers: @happilyhughes\n        latestComments: 😂😂😂\nYup instilling those old fashioned values to ur daughter- CHANGE UR FACE CAUSE OLD WOME

In [ ]:
res = vector_search("what are some good hashtags to use for my reel about sunscreen?", "Beauty_Skincare")

{'postURL': 'https://www.instagram.com/p/C-GCmrEtnmN/', 'content': 'type: Video\n        timestamp: 2024-07-31T16:50:00.000Z\n        caption: Sunscreen \n\n#sunscreen #skincare #skincareroutine\n        hashtags: #sunscreen #skincare #skincareroutine\n        mentions: \n        commentsCount: 0\n        likesCount: 3\n        videoViewCount: 32\n        videoPlayCount: 197\n        ownerUsername: skin_so_fabulous\n        taggedUsers: \n        latestComments: \n        ', 'profileURL': 'https://www.instagram.com/skin_so_fabulous/'}
0.33223795890808105
{'postURL': 'https://www.instagram.com/p/Cuj_tKyLHrw/', 'content': "type: Video\n        timestamp: 2023-07-11T15:45:05.000Z\n        caption: Sunscreen, Sunscreen, Sunscreen. I can't say it enough. Can't stop, won't stop. \n\n#summertime #protectyourskin #eminenceorganicskincare #mineralsunscreen #nochemicals #skincare #skincaretips #antiaging #beauty #beautifulskin\n        hashtags: #summertime #protectyourskin #eminenceorganicskinc

In [ ]:
print(res)

[{'postURL': 'https://www.instagram.com/p/C-GCmrEtnmN/', 'content': 'type: Video\n        timestamp: 2024-07-31T16:50:00.000Z\n        caption: Sunscreen \n\n#sunscreen #skincare #skincareroutine\n        hashtags: #sunscreen #skincare #skincareroutine\n        mentions: \n        commentsCount: 0\n        likesCount: 3\n        videoViewCount: 32\n        videoPlayCount: 197\n        ownerUsername: skin_so_fabulous\n        taggedUsers: \n        latestComments: \n        ', 'profileURL': 'https://www.instagram.com/skin_so_fabulous/'}, {'postURL': 'https://www.instagram.com/p/Cuj_tKyLHrw/', 'content': "type: Video\n        timestamp: 2023-07-11T15:45:05.000Z\n        caption: Sunscreen, Sunscreen, Sunscreen. I can't say it enough. Can't stop, won't stop. \n\n#summertime #protectyourskin #eminenceorganicskincare #mineralsunscreen #nochemicals #skincare #skincaretips #antiaging #beauty #beautifulskin\n        hashtags: #summertime #protectyourskin #eminenceorganicskincare #mineralsunscr

In [42]:
def ingest_user_data(user, collection):
    # Connect to Weaviate Cloud
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=WEAVIATE_URL,
        auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    )
    data = getPosts(user)
    vectors = add_embeddings_to_posts(data)

    collection = client.collections.get(collection)

    with collection.batch.dynamic() as batch:
        for i, data_row in enumerate(data):
            # print(f"i: {i} data_row: {data_row}")
            # print(f"vectors: {vectors[i]}")
            batch.add_object(
                properties=data_row,
                vector=vectors[i]
            )
    return "Data ingested successfully!"

In [43]:
def getPosts(url):
    # Initialize the ApifyClient with API token
    client = ApifyClient(apify_api_key)

    # Prepare the Actor input
    run_input = {
        "addParentData": False,
        "directUrls": [url],
        "enhanceUserSearchWithFacebookPage": True,
        "isUserReelFeedURL": True,
        "isUserTaggedFeedURL": False,
        "resultsLimit": 5,
        "resultsType": "posts",
    }

    posts = []

    # Run the Actor and wait for it to finish
    run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        posts.append(item)
    print("Length of posts: " + str(len(posts)))

    def extractPostData(original_dict):
        # Extract the necessary information from the original dictionary
        extracted_info = {
            'type': original_dict.get('type'),
            'timestamp': original_dict.get('timestamp'),
            'caption': original_dict.get('caption'),
            'hashtags': ' '.join(['#' + tag for tag in original_dict.get('hashtags') or []]),
            'mentions': ' '.join(['@' + mention for mention in original_dict.get('mentions') or []]),
            'url': original_dict.get('url'),
            'videoUrl': original_dict.get('videoUrl') if original_dict.get('type') == 'Video' else None,
            'displayUrl': original_dict.get('displayUrl'),
            'commentsCount': original_dict.get('commentsCount'),
            'likesCount': original_dict.get('likesCount'),
            'videoViewCount': original_dict.get('videoViewCount') if original_dict.get('type') == 'Video' else None,
            'videoPlayCount': original_dict.get('videoPlayCount') if original_dict.get('type') == 'Video' else None,
            'ownerUsername': original_dict.get('ownerUsername'),
            'taggedUsers': ' '.join(['@' + user['username'] for user in original_dict.get('taggedUsers', [])]),
            'latestComments': '\n'.join([comment['text'] for comment in original_dict.get('latestComments', [])])
        }

        # Create a content string to include all the extracted details
        content = f"""type: {extracted_info['type']}
        timestamp: {extracted_info['timestamp']}
        caption: {extracted_info['caption']}
        hashtags: {extracted_info['hashtags']}
        mentions: {extracted_info['mentions']}
        commentsCount: {extracted_info['commentsCount']}
        likesCount: {extracted_info['likesCount']}
        videoViewCount: {extracted_info['videoViewCount']}
        videoPlayCount: {extracted_info['videoPlayCount']}
        ownerUsername: {extracted_info['ownerUsername']}
        taggedUsers: {extracted_info['taggedUsers']}
        latestComments: {extracted_info['latestComments']}
        """

        post_url = extracted_info['url']  # Extract the post's URL

        return content, post_url

    extracted_posts = []
    for post in posts:
        extracted_post, post_url = extractPostData(post)
        extracted_posts.append({"profileURL": url, "postURL": post_url, "content": extracted_post})
    return extracted_posts

In [44]:
def add_embeddings_to_posts(posts_data):
    """
    Adds vector embeddings to each post's content.

    Args:
        posts_data (list of dicts): List of dictionaries with keys: profileURL, postURL, content.

    Returns:
        list of dicts: Updated list with an added "vector" key containing the embeddings.
    """
    # Initialize the Hugging Face Embedding class
    embeddings_model = HuggingFaceEmbeddings()

    vectors = []

    for post in posts_data:
        content = post['content']
        # Generate embeddings for the content
        vector = embeddings_model.embed_query(content)
        # Add the vector embeddings to the dictionary
        vectors.append(vector)

    return vectors

In [46]:
print(getPosts('https://www.instagram.com/saniyafarrukh/'))

Length of posts: 1
[{'profileURL': 'https://www.instagram.com/saniyafarrukh/', 'postURL': None, 'content': 'type: None\n        timestamp: None\n        caption: None\n        hashtags: \n        mentions: \n        commentsCount: None\n        likesCount: None\n        videoViewCount: None\n        videoPlayCount: None\n        ownerUsername: None\n        taggedUsers: \n        latestComments: \n        '}]


In [45]:
ingest_user_data('https://www.instagram.com/saniyafarrukh/', 'Beauty')

Length of posts: 1


'Data ingested successfully!'